In [2]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et 

In [3]:
xml_list = glob('./data/*.xml')

In [4]:
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    image_name = root.find('filename').text
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')

    parser = []
    for o in objs:
        name = o.find('name').text
        bndbox = o.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name, xmin, xmax, ymin, ymax])

    return parser

In [5]:
parser_all = list(map(extract_text, xml_list))

data = reduce(lambda x, y: x + y, parser_all)

In [6]:
df = pd.DataFrame(data, columns=['filename', 'width', 'height', 'name', 'xmin', 'xmax', 'ymin', 'ymax'])

In [7]:
print(df.shape, df['name'].value_counts())

(335, 8) up      194
down    141
Name: name, dtype: int64


In [8]:
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']
df[cols] = df[cols].astype(int)

In [9]:
df['center_x'] = (df['xmin'] + df['xmax']) / (2 * df['width'])
df['center_y'] = (df['ymin'] + df['ymax']) / (2 * df['height'])
df['w'] = (df['xmax'] - df['xmin']) / df['width']
df['h'] = (df['ymax'] - df['ymin']) / df['height']

In [10]:
images = df['filename'].unique()
img_df = pd.DataFrame(images, columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename'])
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [11]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [13]:
def label_encoding(x):
    labels = {'down': 0, 'up': 1}
    return labels[x]

In [14]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

/tmp/ipykernel_5129/3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
/tmp/ipykernel_5129/3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [16]:
from shutil import move
import os

In [18]:
train_folder = 'org_data/train'
test_folder = 'org_data/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

In [20]:
cols = ['filename', 'id', 'center_x', 'center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [21]:
def save_data(filename, folder_path, group_obj):
    src = os.path.join('data', filename)
    dst = os.path.join(folder_path, filename)
    move(src, dst)

    text_filename = os.path.join(folder_path, os.path.splitext(filename)[0] + '.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename, sep=' ', index=False, header=False)

In [22]:
filename_series = pd.Series(groupby_obj_train.groups.keys())
filename_series.apply(save_data, args=(train_folder, groupby_obj_train))

0      None
1      None
2      None
3      None
4      None
       ... 
207    None
208    None
209    None
210    None
211    None
Length: 212, dtype: object

In [23]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data, args=(test_folder, groupby_obj_test))

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
40    None
41    None
42    None
43    None
44    None
45    None
46    None
47    None
48    None
49    None
50    None
51    None
52    None
dtype: object